In [26]:
import psycopg2

try:
    connection = psycopg2.connect(
        user="user",
        port="port",
        host="localhost",
        database="name"
    )
    
    cursor = connection.cursor()
    
    # The table sentiment_analysis_data_nvidia contains tens of thousands of news articles about NVDA
    select_query = "SELECT * FROM sentiment_analysis_data_nvidia;"

    cursor.execute(select_query)

    # Fetch all records
    records_nvidia = cursor.fetchall()
    print(records_nvidia[700][3])
    
except Exception as error:
    print(f"Failed to fetch records: {error}")

finally:
    # Close the connection
    if connection:
        cursor.close()
        connection.close()
        print("Connection closed.")

bc-IL-7-Summits-alert   02-14 1/8STK 3/8 INTC NVDA BSC DE PCU 1/8IN 3/8 FIN 1/8SU 3/8 INO TO BUSINESS EDITOR: Seven Summits Research Releases Alerts on INTC, NVDA, BSC, DE, and PCU CHICAGO, Feb. 14 /PRNewswire/ -- Seven Summits Research issues PriceWatch Alerts for key stocks. Seven Summits Strategic Investments' PriceWatch Alerts are available at http://www.iotogo.com/smu.asp?p021407I (Note: You may have to copy this link into your browser then press the  1/8ENTER 3/8 key.) Today's PriceWatch Alerts cover the following stocks: Intel Corp. (Nasdaq: INTC), NVIDIA Corp. (Nasdaq: NVDA), Bear Stearns Companies Inc. (NYSE: BSC), Deere & Co. (NYSE: DE), and Southern Copper Corp. (NYSE: PCU). Get Bluewater Asset Managements top 7 stock picks for 2007 http://www.iotogo.com/BWAHedge8 Along with our PriceWatch Alerts, these brief reports contain a concise market overview, economic calendar and Dynamic Market Opportunities. PriceWatch Alerts include hedged trade ideas designed to potentially prot

In [27]:
connection = psycopg2.connect(
    user="user",
    port="port",
    host="localhost",
    database="name"
)

cursor = connection.cursor()

drop_table_query = "DROP TABLE IF EXISTS sentiment_data_nvidia;"
cursor.execute(drop_table_query)
connection.commit()
print("Table dropped successfully.")

create_table_query = '''CREATE TABLE sentiment_data_nvidia (
    ID SERIAL PRIMARY KEY,
    Title TEXT,
    Time_Published TEXT,
    Sentiment_Score REAL
); '''

cursor.execute(create_table_query)
connection.commit()
print("Table Created Successfully")

Table dropped successfully.
Table Created Successfully


In [29]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from nltk.tokenize import sent_tokenize
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

connection = psycopg2.connect(
    user="user",
    port="port",
    host="localhost",
    database="name"
)

cursor = connection.cursor()

# Load the model and tokenizer
model_name = "nickmuchi/deberta-v3-base-finetuned-finance-text-classification"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)  # Moved to device
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocessing function
def preprocess(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return inputs

# Inference function for a single sentence
def get_sentiment(text):
    inputs = preprocess(text).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    label_id = torch.argmax(probabilities).item()
    return label_id

# Your articles
article_text = [record[3] for record in records_nvidia]
article_date = [record[2] for record in records_nvidia]
article_title = [record[1] for record in records_nvidia]

# Iterate over articles
for i, (article, date, title) in enumerate(zip(article_text, article_date, article_title)):
    sentences = sent_tokenize(article)  # Split the article into sentences
    sentiment_scores = [get_sentiment(sentence) for sentence in sentences]  # Get sentiment for each sentence
    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)  # Average the sentiment scores
    cursor.execute("INSERT INTO sentiment_data_nvidia (ID, Title, Sentiment_Score, Time_Published) VALUES (%s, %s, %s, %s)",
                   (i, title, avg_sentiment, date))
    connection.commit()
    print(f"Average Sentiment of Article {i+1} is {avg_sentiment} at {date} for {title}")



Average Sentiment of Article 1 is 1.0204081632653061 at January 8, 2009 Thursday for TheSUBWAY.com Inc.: Market Alerts for Thursday, January 08, 2009: AITG Completes Major Acquisition!
Average Sentiment of Article 2 is 1.0 at January 28, 2009 Wednesday 2:36 PM EST for Make To-Dos Dead Simple With Deadline
Average Sentiment of Article 3 is 0.8620689655172413 at January 2, 2009 Friday 12:15 PM EST for Zune doomsday?
Average Sentiment of Article 4 is 0.9555555555555556 at January 12, 2009 Monday 9:01 AM EST for Citadel Investment Group (Ken Griffin): Hedge Fund Portfolio Tracking - 13F
Average Sentiment of Article 5 is 0.76 at January 14, 2009 Wednesday 4:17 PM EST for Would Apportionment of Damages Adversely Effect Patents?
Average Sentiment of Article 6 is 0.8787878787878788 at January 23, 2009 Friday 4:27 AM EST for Obama staffers ignorant of tech security, public records law
Average Sentiment of Article 7 is 1.1 at January 23, 2009 Friday 3:05 PM GMT for BullMarket.com Updates Outlook